In [10]:
import pandas as pd
import os
import numpy as np

In [57]:
path = '/Users/chunyuewei/Desktop/CS 229/Project/CS229/Example Data'
csv_name = os.path.join(path + os.sep + 'one_day_2018-06-01_2018-06-02.csv')

In [58]:
df = pd.read_csv(csv_name)

### Code

In [60]:
codes_inline = []
codes_pre_line = []
codes_pre_count = []
for i in range(df.shape[0]):
    body = df['Body'][i]
    code_start = 0
    code_end = 0
    code_pre_line = 0
    code_pre_count = 0
    code_inline = 0
    while True:
        code_start = body.find('<code>', code_end)
        if code_start == -1:
            break
        code_end = body.find('</code>', code_start)
        code = body[code_start: code_end+7]
        if body[code_start-5: code_start] == '<pre>':
            code_line = code.split('\n')
            # if '' in code_line: # Remove blank lines
                # while '' in code_line:
                #    code_line.remove('')    
            code_line_count = len(code_line)-1 # Last line as </code>
            code_pre_line += code_line_count
            code_pre_count += 1
        else:
            code_inline += 1
    codes_inline.append(code_inline)
    codes_pre_line.append(code_pre_line)
    codes_pre_count.append(code_pre_count)

In [61]:
df['InlineCode'] = codes_inline
df['BlockCode'] = codes_pre_count
df['BlockCodeLine'] = codes_pre_line

### Hyperlink

In [62]:
hyperlinks = []
for i in range(df.shape[0]):
    body = df['Body'][i]
    hyperlink_pos = -1
    hyperlink = 0
    while True:
        hyperlink_pos = body.find('<a href=', hyperlink_pos+1)
        if hyperlink_pos == -1:
            break
        hyperlink += 1
        # print(i, body[hyperlink_pos:hyperlink_pos + 50])
    hyperlinks.append(hyperlink)

In [63]:
df['Hyperlink'] = hyperlinks

### Edit and Label

In [64]:
edits = []
labels = []
for i in range(df.shape[0]):
    last_edit_date = df['LastEditDate'][i] # str or float (np.nan)
    if isinstance(last_edit_date, float):
        if np.isnan(last_edit_date):
            edits.append('0')
        else:
            edits.append('1')
    else:
        edits.append('1')
        
    answer_id = df['Id'][i] # int
    accepted_id = df['ParentAcceptedAnswerId'][i] # float (np.nan)
    if np.isnan(accepted_id):
        labels.append(np.nan)
    else:
        if answer_id == int(accepted_id):
            labels.append('1')
        else:
            labels.append('0')

In [65]:
df['Edit'] = edits
df['Label'] = labels

### Output data

In [67]:
output = df[['Label','Edit','CommentCount','BodyLength', 'UserReputation','UserViews','UserUpVotes','UserDownVotes','InlineCode','BlockCode','BlockCodeLine','Hyperlink']]

In [68]:
output_name = csv_name.replace('.csv','_input.csv')
output.to_csv(output_name)

## Text